# Analysis of Paleo-present Climate Data
***

## Background

## Data Collection and Loading

## Data Processing

## Data Analysis
### CO2 vs Temperature Anomaly Analysis

### Additional Feature Analysis

### Irish Context Analysis

## Data Export

## Temperature Anomaly Prediction

## Conclusion

## References

***
# End